# Import Modules

In [66]:
import os
import pandas as pd
import numpy.random as rnd
from dotenv import load_dotenv
import requests
import warnings
import json
import sqlalchemy as sql
from pathlib import Path
import datetime
import yfinance as yf
import numpy as np


from MCForecastTools import MCSimulation
warnings.filterwarnings('ignore')


%matplotlib inline

#warnings.filterwarnings("ignore")
#pd.options.display.float_format = '{:.4%}'.format

# Client Input Module

# Data Input for Universe (Multiple Asset Classes)

In [67]:
# Date range
today = datetime.date.today()
start_date = today - datetime.timedelta(days=365*5) #trading days(252) * amount of years to go back 

start = start_date.strftime("%Y-%m-%d")
end = today.strftime("%Y-%m-%d")

# Tickers of assets
assets = ["XLC", "XLY", "XLP", "XLE", "XLF", "XLV", "XLI", "XLB", "XLRE", "XLK", "XLU", "AAPL", "MSFT", "NVDA", "AMZN", "BRK-B",
         "GOOG", "META", "UNH", "XOM", "AGG", "BND", "LQD", "VCIT", "BNDX", "TMF", "TLT", "ICVT", "LKOR", "FBND"]
assets.sort()

# Downloading data
og_data = yf.download(assets, start = start, end = end)
data = og_data.loc[:,('Adj Close', slice(None))]
data.columns = assets

Y = data[assets].pct_change().dropna()

display(start)
display(end)
display(Y)

[*********************100%***********************]  30 of 30 completed


'2018-06-30'

'2023-06-29'

,AAPL,AGG,AMZN,BND,BNDX,BRK-B,FBND,GOOG,ICVT,LKOR,...,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2018-07-03,-0.017416,0.002076,-0.011565,0.002027,0.000183,-0.009636,0.000615,-0.021792,-0.002304,-0.000201,...,0.006282,-0.008982,-0.003204,-0.011842,0.002342,0.005842,0.002866,0.002389,-0.005025,0.005871
2018-07-05,0.008047,0.000942,0.003406,0.001390,0.001279,0.007956,0.001228,0.019385,0.006947,0.006239,...,-0.001063,0.002266,0.004891,0.014149,0.013827,0.013146,0.005335,0.010367,0.004959,0.000851
2018-07-06,0.013862,0.001035,0.006413,0.001136,0.000000,0.000320,0.001636,0.014142,0.009141,0.002000,...,0.006116,0.004898,0.003477,0.011674,0.003074,0.004225,0.007771,0.014389,0.007858,0.000243
2018-07-09,0.013885,-0.000939,0.016596,-0.000882,0.000730,0.011090,-0.001021,0.012174,0.004102,0.003992,...,0.014933,0.022872,0.018570,0.007599,-0.003830,-0.009315,-0.030844,0.007325,0.011967,0.006923
2018-07-10,-0.001207,-0.000753,0.002329,-0.000631,-0.000547,0.001371,-0.000817,-0.001049,-0.000171,0.000000,...,0.007422,-0.003666,0.003129,0.002374,0.011919,0.004853,0.009509,0.003809,0.002150,0.009168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-22,0.016525,-0.005179,0.042618,-0.004798,-0.003896,-0.004873,-0.003504,0.021524,0.000398,-0.008210,...,-0.013421,-0.007845,-0.007250,0.008160,0.005823,-0.014142,-0.007668,0.006842,0.012766,-0.005488
2023-06-23,-0.001711,0.002960,-0.006300,0.002893,0.005146,-0.005075,0.001319,-0.006862,-0.007294,0.004730,...,-0.007187,-0.004258,-0.007591,-0.010850,-0.006598,-0.010759,-0.014697,-0.003322,-0.011590,-0.008713
2023-06-26,-0.007553,0.001221,-0.015464,0.001236,0.002662,-0.003371,0.002414,-0.031946,0.001336,-0.000069,...,0.017968,-0.002138,0.008327,-0.007175,0.001355,0.022309,0.009842,-0.005682,-0.012814,0.018457


In [68]:
Y_cumulative_ret = (1 + Y).cumprod()
Y_cumulative_ret

,AAPL,AGG,AMZN,BND,BNDX,BRK-B,FBND,GOOG,ICVT,LKOR,...,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2018-07-03,0.982584,1.002076,0.988435,1.002027,1.000183,0.990364,1.000615,0.978208,0.997696,0.999799,...,1.006282,0.991018,0.996796,0.988158,1.002342,1.005842,1.002866,1.002389,0.994975,1.005871
2018-07-05,0.990490,1.003019,0.991802,1.003420,1.001462,0.998243,1.001844,0.997171,1.004627,1.006036,...,1.005212,0.993264,1.001672,1.002140,1.016201,1.019065,1.008217,1.012781,0.999909,1.006727
2018-07-06,1.004220,1.004057,0.998162,1.004560,1.001462,0.998563,1.003482,1.011273,1.013810,1.008049,...,1.011360,0.998129,1.005154,1.013839,1.019325,1.023370,1.016052,1.027353,1.007766,1.006972
2018-07-09,1.018164,1.003114,1.014728,1.003673,1.002193,1.009636,1.002458,1.023584,1.017968,1.012073,...,1.026463,1.020958,1.023820,1.021544,1.015420,1.013837,0.984712,1.034878,1.019826,1.013943
2018-07-10,1.016936,1.002358,1.017091,1.003040,1.001645,1.011021,1.001639,1.022511,1.017795,1.012073,...,1.034082,1.017216,1.027023,1.023969,1.027523,1.018758,0.994076,1.038820,1.022019,1.023239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-22,4.181497,1.040235,1.518865,1.040795,1.008656,1.793963,1.082394,2.197329,1.591626,1.070890,...,1.340434,1.365733,1.586206,2.570115,1.656659,1.319317,1.478465,1.724768,1.609856,1.636726
2023-06-23,4.174341,1.043314,1.509295,1.043806,1.013846,1.784859,1.083821,2.182250,1.580016,1.075956,...,1.330801,1.359918,1.574165,2.542228,1.645729,1.305123,1.456736,1.719038,1.591198,1.622466
2023-06-26,4.142812,1.044588,1.485955,1.045096,1.016545,1.778843,1.086438,2.112536,1.582127,1.075881,...,1.354712,1.357010,1.587273,2.523989,1.647960,1.334239,1.471072,1.709271,1.570809,1.652412


In [69]:
#Set the time period and trading days
time_period = 1
trading_days = 252 * time_period

In [70]:
# Calculate and sort the standard deviation for all assetts
asset_standard_deviation = pd.DataFrame(Y.std(), columns = ['Standard Deviation'], index=assets)
asset_standard_deviation

,Standard Deviation
AAPL,0.021004
AGG,0.003865
AMZN,0.022962
BND,0.004022
BNDX,0.002902
BRK-B,0.014257
FBND,0.004566
GOOG,0.020146
ICVT,0.012057
LKOR,0.009798


In [71]:
# Calculate and sort the annualized standard deviation (252 trading days) 
annual_stdev_calc = Y.std()*np.sqrt(trading_days)
annual_standard_deviation = pd.DataFrame(annual_stdev_calc, columns = ['Annual Standard Deviation'], index=assets)
display(annual_standard_deviation)

,Annual Standard Deviation
AAPL,0.333433
AGG,0.061350
AMZN,0.364503
BND,0.063855
BNDX,0.046075
BRK-B,0.226326
FBND,0.072482
GOOG,0.319809
ICVT,0.191399
LKOR,0.155541


In [72]:
annualized_avg_return_calc = Y.mean() * trading_days
annualized_average_returns = pd.DataFrame(annualized_avg_return_calc, columns = ['Annual Average Returns'], index=assets)
annualized_average_returns

,Annual Average Returns
AAPL,0.345434
AGG,0.010851
AMZN,0.148599
BND,0.011149
BNDX,0.004562
BRK-B,0.141300
FBND,0.019409
GOOG,0.204651
ICVT,0.112422
LKOR,0.027608


In [73]:
# Calculate the annualized Sharpe Ratios
Sharpe_ratios_calc = annualized_average_returns['Annual Average Returns'] / annual_standard_deviation['Annual Standard Deviation']
Sharpe_ratios = pd.DataFrame(Sharpe_ratios_calc, columns = ['Sharpe Ratio'], index=assets)
Sharpe_ratios

,Sharpe Ratio
AAPL,1.035991
AGG,0.176869
AMZN,0.407676
BND,0.174598
BNDX,0.099012
BRK-B,0.624320
FBND,0.267774
GOOG,0.639917
ICVT,0.587371
LKOR,0.177500


In [74]:
# Merge all of the dataframes
Asset_Quant_Data = pd.merge(pd.merge(pd.merge           
                        (asset_standard_deviation, annual_standard_deviation, left_index=True, right_index=True),
                        annualized_average_returns, left_index=True, right_index=True), 
                        Sharpe_ratios, left_index=True, right_index=True)
Asset_Quant_Data

,Standard Deviation,Annual Standard Deviation,Annual Average Returns,Sharpe Ratio
AAPL,0.021004,0.333433,0.345434,1.035991
AGG,0.003865,0.061350,0.010851,0.176869
AMZN,0.022962,0.364503,0.148599,0.407676
BND,0.004022,0.063855,0.011149,0.174598
BNDX,0.002902,0.046075,0.004562,0.099012
BRK-B,0.014257,0.226326,0.141300,0.624320
FBND,0.004566,0.072482,0.019409,0.267774
GOOG,0.020146,0.319809,0.204651,0.639917
ICVT,0.012057,0.191399,0.112422,0.587371
LKOR,0.009798,0.155541,0.027608,0.177500
